In [0]:
-- Things that we need in Unpaywall but do not want in OpenAlex should be filtered out in the openalex temp view, created below.
-- create or replace temp view openalex as (
--   select id, doi, title, authorships, publication_date, publication_year, abstract, has_abstract, abstract_inverted_index, ids, language, publisher, apc_list, locations_count from openalex.works.openalex_works -- filter out crossref component types
-- )

CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works.openalex_works_json') CLUSTER BY (id) AS (

  WITH transformed AS (
    SELECT
      CONCAT('https://openalex.org/W', id) AS id,
      doi,
      title,
      authorships,
      publication_date,
      publication_year,
      abstract,
      has_abstract,
      referenced_works_count,
      SORT_ARRAY(TRANSFORM(referenced_works, rw -> CONCAT('https://openalex.org/W', rw))) AS referenced_works,
      abstract_inverted_index,
      cited_by_api_url,
      cited_by_count,
      counts_by_year,
      ids,
      language,
      publisher,
      biblio,
      apc_list,
      locations_count,
      locations,
      primary_location,
      best_oa_location,
      open_access,
      type,
      is_paratext,
      is_retracted,
      indexed_in_crossref,
      mesh,
      created_date,
      updated_date
    FROM identifier('openalex' || :env_suffix || '.works.openalex_works')
  )

  SELECT 
    id,
    TO_JSON(STRUCT(*)) AS json_str
  FROM transformed

);


In [0]:
OPTIMIZE identifier('openalex' || :env_suffix || '.works.openalex_works_json');

In [0]:
SELECT * FROM identifier('openalex' || :env_suffix || '.works.openalex_works_json') where id = 'https://openalex.org/W1775749144'

In [0]:
-- should we truncate the fulltext field in "Sync Works to ElasticSearch"? 13M
select format_number(count(*),0) as too_long_count
from openalex.works.openalex_works where length(fulltext) > 32766